### Comparing model with BERT model (RoBERTa)





In [7]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [9]:
# Kiểm tra nếu GPU khả dụng, nếu không thì dùng CPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# Mount Google Drive để truy cập dữ liệu
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Code/Notebooks

/content/drive/MyDrive/Code/Notebooks


In [ ]:
# Đọc tập dữ liệu huấn luyện từ Google Drive 
train = pd.read_csv("/content/drive/MyDrive/final_train_df.csv", encoding = "ISO-8859-1")
train = train.drop(train.columns[0], axis=1) # Bỏ cột không cần thiết
train.head()# Hiển thị vài dòng đầu tiên của tập dữ liệu

In [ ]:
# Tạo một DataFrame mới chỉ chứa cột văn bản và nhãn
new_df = train[['text', 'airline_sentiment']].copy()
# Chuyển nhãn từ dạng chuỗi thành số (positive = 0, neutral = 1, negative = 2)
new_df['airline_sentiment'].replace(['positive', 'neutral', 'negative'], [0, 1, 2], inplace=True)

In [ ]:
# Định nghĩa các tham số cho mô hình
MAX_LEN = 256  # Độ dài tối đa của chuỗi sau tokenization
TRAIN_BATCH_SIZE = 8  # Kích thước batch trong tập train
VALID_BATCH_SIZE = 4  # Kích thước batch trong tập validation
LEARNING_RATE = 1e-05  # Tốc độ học
# Khởi tạo tokenizer từ mô hình RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
# Định nghĩa class xử lý dữ liệu
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer  # Tokenizer để xử lý văn bản
        self.data = dataframe  # Dữ liệu
        self.text = dataframe.text  # Cột chứa văn bản
        self.targets = self.data.airline_sentiment  # Nhãn mục tiêu
        self.max_len = max_len  # Độ dài tối đa

    def __len__(self):
        return len(self.text)   # Trả về số lượng mẫu

    def __getitem__(self, index):
        # Lấy văn bản tại một chỉ số cụ thể và làm sạch
        text = str(self.text[index])
        text = " ".join(text.split())

        # Áp dụng tokenizer để chuyển văn bản thành input IDs và attention mask
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']  # Input IDs
        mask = inputs['attention_mask']  # Attention mask
        token_type_ids = inputs["token_type_ids"]  # Token type IDs

        # Trả về dictionary chứa các input và nhãn mục tiêu
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Chia dữ liệu thành tập train và test (80% train, 20% test)
train_size = 0.8
train_data = new_df.sample(frac=train_size, random_state=733)  # Lấy ngẫu nhiên tập train
test_data = new_df.drop(train_data.index).reset_index(drop=True)  # Tập test là phần còn lại
train_data = train_data.reset_index(drop=True)

# Khởi tạo tập dữ liệu train và test
training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

In [ ]:
# Tạo DataLoader để lặp qua dữ liệu theo batch
training_loader = DataLoader(training_set, batch_size = 8)
testing_loader = DataLoader(testing_set, batch_size = 4)

In [ ]:
# Định nghĩa class mô hình RoBERTa
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")  # Sử dụng mô hình RoBERTa gốc
        self.pre_classifier = torch.nn.Linear(768, 768)  # Lớp dense trước phân loại
        self.dropout = torch.nn.Dropout(0.3)  # Dropout để giảm overfitting
        self.classifier = torch.nn.Linear(768, 5)  # Lớp đầu ra (5 nhãn)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # Trích xuất các đặc trưng từ RoBERTa
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]  # Lấy hidden state
        pooler = hidden_state[:, 0]  # Lấy embedding của token [CLS]
        pooler = self.pre_classifier(pooler)  # Qua lớp dense
        pooler = torch.nn.ReLU()(pooler)  # Áp dụng ReLU
        pooler = self.dropout(pooler)  # Dropout
        output = self.classifier(pooler)  # Lớp phân loại cuối cùng
        return output

In [ ]:
# Hàm tính độ chính xác
def get_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()   # Số lượng dự đoán đúng
    return n_correct

In [ ]:
# Hàm huấn luyện mô hình
def train(epoch):
    tr_loss = 0  # Tổng loss
    n_correct = 0  # Số lượng dự đoán đúng
    nb_tr_steps = 0  # Số bước lặp
    nb_tr_examples = 0  # Số mẫu đã huấn luyện
    model.train()  # Chuyển mô hình sang chế độ train
    for _, data in tqdm(enumerate(training_loader, 0)):  # Lặp qua tập dữ liệu train
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.long)

        outputs = model(ids, mask, token_type_ids)  # Lấy dự đoán từ mô hình
        loss = loss_function(outputs, targets)  # Tính loss
        tr_loss += loss.item()  # Cộng dồn loss
        big_val, big_idx = torch.max(outputs.data, dim=1)  # Lấy nhãn dự đoán
        n_correct += get_accuracy(big_idx, targets)  # Cộng số dự đoán đúng

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if _ % 5000 == 0:  # Cập nhật log sau mỗi 5000 batch
            loss_step = tr_loss / nb_tr_steps
            accu_step = (n_correct * 100) / nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()  # Reset gradient
        loss.backward()  # Lan truyền ngược
        optimizer.step()  # Cập nhật trọng số

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct * 100) / nb_tr_examples}')
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")
    return

In [ ]:
# Khởi tạo mô hình RoBERTa
model = RobertaClass().to(device)

# Tạo hàm loss và optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE

In [ ]:
# Huấn luyện mô hình
for epoch in range(0,1):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 5000 steps: 1.6126948595046997
Training Accuracy per 5000 steps: 0.0


1548it [18:13,  1.42it/s]

The Total Accuracy for Epoch 0: 81.40248828566813
Training Loss Epoch: 0.5012302471418656
Training Accuracy Epoch: 81.40248828566813


In [ ]:
# Hàm đánh giá mô hình trên tập test
def valid(model, testing_loader):
    model.eval()  # Chuyển mô hình sang chế độ đánh giá
    n_correct = 0
    n_wrong = 0
    total = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    with torch.no_grad():  # Không tính gradient
        for _, data in tqdm(enumerate(testing_loader, 0)):  # Lặp qua tập test
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)  # Tính loss
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)  # Lấy nhãn dự đoán
            n_correct += get_accuracy(big_idx, targets)  # Cộng số dự đoán đúng

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:  # Log mỗi 5000 batch
                loss_step = tr_loss / nb_tr_steps
                accu_step = (n_correct * 100) / nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [ ]:
# Đánh giá mô hình và in độ chính xác
acc = valid(model, testing_loader)
print(f'Accuracy on test data = {acc}%')

1it [00:00,  8.57it/s]

Validation Loss per 100 steps: 1.1415438652038574
Validation Accuracy per 100 steps: 50.0


774it [01:38,  7.89it/s]

Validation Loss Epoch: 0.4418863855742727
Validation Accuracy Epoch: 83.9043309631545
Accuracy on test data = 83.90%


Training RoBERTa model with 1 epoch (due to GPU constraint) on same airline data we found that training accuracy was 81.42% and validation accuracy is 83.90%.